# Homework 04 - Applied ML

## Question 1: Propensity score matching

In this exercise, you will apply [propensity score matching](http://www.stewartschultz.com/statistics/books/Design%20of%20observational%20studies.pdf), which we discussed in lecture 5 ("Observational studies"), in order to draw conclusions from an observational study.

We will work with a by-now classic dataset from Robert LaLonde's study "[Evaluating the Econometric Evaluations of Training Programs](http://people.hbs.edu/nashraf/LaLonde_1986.pdf)" (1986).
The study investigated the effect of a job training program ("National Supported Work Demonstration") on the real earnings of an individual, a couple of years after completion of the program.
Your task is to determine the effectiveness of the "treatment" represented by the job training program.


#### Dataset description

- `treat`: 1 if the subject participated in the job training program, 0 otherwise
- `age`: the subject's age
- `educ`: years of education
- `race`: categorical variable with three possible values: Black, Hispanic, or White
- `married`: 1 if the subject was married at the time of the training program, 0 otherwise
- `nodegree`: 1 if the subject has earned no school degree, 0 otherwise
- `re74`: real earnings in 1974 (pre-treatment)
- `re75`: real earnings in 1975 (pre-treatment)
- `re78`: real earnings in 1978 (outcome)

If you want to brush up your knowledge on propensity scores and observational studies, we highly recommend Rosenbaum's excellent book on the ["Design of Observational Studies"](http://www.stewartschultz.com/statistics/books/Design%20of%20observational%20studies.pdf). Even just reading the first chapter (18 pages) will help you a lot.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import linear_model
import networkx as nx
%matplotlib inline

In [ ]:
dataset = pd.read_csv('lalonde.csv')
dataset.head()

#### 1. A naive analysis

Compare the distribution of the outcome variable (`re78`) between the two groups, using plots and numbers.
To summarize and compare the distributions, you may use the techniques we discussed in lectures 4 ("Read the stats carefully") and 6 ("Data visualization").

What might a naive "researcher" conclude from this superficial analysis?

## Answer
We divide the dataset into two groups: treated and non-treated (according to the indicator variable *treat*), and we try to evaluate whether there are differences in earning on the outcome variable (*re78*). We start by plotting the distributions of the two groups.
TODO: write about zeros
TODO: write about fitting and histograms

In [ ]:
outcome_treated = dataset[dataset['treat'] == 1]['re78']
outcome_nontreated = dataset[dataset['treat'] == 0]['re78']
bins = np.linspace(dataset['re78'].min(), dataset['re78'].max(), 30)
sns.distplot(outcome_treated, kde=False, fit=stats.expon, bins=bins)
sns.distplot(outcome_nontreated, kde=False, fit=stats.expon, bins=bins)

It appears that the two distributions are heavy-tailed and do not differ significantly.
We now have a look at the data and see if this is the case. Since we are dealing with a heavy-tailed distribution, computing the mean does not make much sense. Therefore, we will analyze the median.

In [ ]:
print('Number of treated individuals:', len(outcome_treated))
print('Number of non-treated individuals:', len(outcome_nontreated))
print('Treated earnings (median):', outcome_treated.median())
print('Non-treated earnings (median):', outcome_nontreated.median())

The non-treated group seems to have higher earnings than the treated group. However, we have to argue whether this difference is significant. In addition, we can see that the population sizes are unbalanced: we have more individuals in the non-treatment group (429) than the treatment group (185).

To make our assumption stronger, we run an hypothesis test where we aim to reject the null hypothesis $H_0$, which states that the two outcomes are drawn from the same probability distribution. As can be seen from the histograms above, the distributions are certainly not Gaussian, but resemble a power law.
We decided to adopt the K-S (Kolmogorov-Smirnov) test, which is a non-parametric test, and as such does not require the assumption that the data is normally-distributed. We are interested in the two-sided test, which is used for comparing two distribution.

In [ ]:
stats.ks_2samp(outcome_nontreated, outcome_treated)

As can be seen, the p-value is not low enough to reject the null hypothesis. We are assuming a threshold of $p = 0.05$ for the rejection. We can conclude that there is not sufficient evidence that the treatment is effective in improving the income.

#### 2. A closer look at the data

You're not naive, of course (and even if you are, you've learned certain things in ADA), so you aren't content with a superficial analysis such as the above.
You're aware of the dangers of observational studies, so you take a closer look at the data before jumping to conclusions.

For each feature in the dataset, compare its distribution in the treated group with its distribution in the control group, using plots and numbers.
As above, you may use the techniques we discussed in class for summarizing and comparing the distributions.

What do you observe?
Describe what your observations mean for the conclusions drawn by the naive "researcher" from his superficial analysis.

In [ ]:
def plot_features(dataset):
    n_cols, n_rows = 2, 4
    width, heigth = n_cols*6, n_rows*5
    
    fig, axs = plt.subplots(n_rows, n_cols)
    fig.set_size_inches(width, heigth)
    
    treated = dataset[dataset['treat'] == 1]
    nontreated = dataset[dataset['treat'] == 0]
    
    continuous = ['age', 'educ', 're74', 're75']
    for i, feature in enumerate(continuous):
        r, c = int(i/2), i%2
        sns.distplot(treated[feature], ax=axs[r][c])
        sns.distplot(nontreated[feature], ax=axs[r][c])
        axs[r][c].legend(['Treated', 'Non-treated'])
        axs[r][c].set_ylabel('normed occurrencies')

    discretes = ['black', 'hispan', 'married', 'nodegree']
    contrary = {'black': 'white', 'hispan': 'not hispan', 'married': 'unmarried', 'nodegree': 'with degree'}
    for i, feature in enumerate(discretes):
        r, c = int(i/2)+2, i%2
        agg = dataset.groupby(['treat'])[feature].count()
        (dataset.groupby(['treat', feature])[feature].count()/agg).unstack().plot.bar(stacked=True, ax=axs[r][c])
        
        axs[r][c].legend([contrary[feature], feature])
        axs[r][c].set_ylabel('ratio')
        
    plt.tight_layout()
    

In [ ]:
plot_features(dataset)

In [ ]:
def compare(variable):
    treated = dataset[dataset['treat'] == 1][variable]
    nontreated = dataset[dataset['treat'] == 0][variable]
    sns.distplot(treated)
    ax = sns.distplot(nontreated)
    ax.legend(['Treated', 'Non-treated'])
    
def compare_categorical(variable):
    agg = dataset.groupby(['treat'])[variable].count()
    ax = (dataset.groupby(['treat', variable])[variable].count()/agg).unstack().plot.bar(stacked=True)

In [ ]:
compare('age')

In [ ]:
compare('educ')

In [ ]:
compare('re74')

In [ ]:
compare('re75')

In [ ]:
compare_categorical('black')

In [ ]:
compare_categorical('hispan')

In [ ]:
compare_categorical('married')

In [ ]:
compare_categorical('nodegree')

#### 3. A propensity score model

Use logistic regression to estimate propensity scores for all points in the dataset.
You may use `sklearn` to fit the logistic regression model and apply it to each data point to obtain propensity scores:

```python
from sklearn import linear_model
logistic = linear_model.LogisticRegression()
```

Recall that the propensity score of a data point represents its probability of receiving the treatment, based on its pre-treatment features (in this case, age, education, pre-treatment income, etc.).
To brush up on propensity scores, you may read chapter 3.3 of the above-cited book by Rosenbaum or [this article](https://drive.google.com/file/d/0B4jctQY-uqhzTlpBaTBJRTJFVFE/view).

Note: you do not need a train/test split here. Train and apply the model on the entire dataset. If you're wondering why this is the right thing to do in this situation, recall that the propensity score model is not used in order to make predictions about unseen data. Its sole purpose is to balance the dataset across treatment groups.
(See p. 74 of Rosenbaum's book for an explanation why slight overfitting is even good for propensity scores.
If you want even more information, read [this article](https://drive.google.com/file/d/0B4jctQY-uqhzTlpBaTBJRTJFVFE/view).)

In [ ]:
logistic = linear_model.LogisticRegression(C=1e9)
X = dataset[['age', 'educ', 'black', 'hispan', 'married', 'nodegree', 're74', 're75']]
y = dataset['treat']
logistic.fit(X, y)
dataset['propensity'] = logistic.predict_proba(X)[:, 1]

logistic.score(X, y)

In [ ]:
dataset.head()

#### 4. Balancing the dataset via matching

Use the propensity scores to match each data point from the treated group with exactly one data point from the control group, while ensuring that each data point from the control group is matched with at most one data point from the treated group.
(Hint: you may explore the `networkx` package in Python for predefined matching functions.)

Your matching should maximize the similarity between matched subjects, as captured by their propensity scores.
In other words, the sum (over all matched pairs) of absolute propensity-score differences between the two matched subjects should be minimized.

After matching, you have as many treated as you have control subjects.
Compare the outcomes (`re78`) between the two groups (treated and control).

Also, compare again the feature-value distributions between the two groups, as you've done in part 2 above, but now only for the matched subjects.
What do you observe?
Are you closer to being able to draw valid conclusions now than you were before?

In [ ]:
treated = dataset[dataset['treat'] == 1].index
nontreated = dataset[dataset['treat'] == 0].index

G = nx.Graph()

for i in dataset.index:
    G.add_node(i)
    
for i in treated:
    for j in nontreated:
        G.add_edge(i, j, weight=-abs(dataset['propensity'].loc[i] - dataset['propensity'].loc[j]))

In [ ]:
matching = nx.algorithms.max_weight_matching(G, maxcardinality=True)

In [ ]:
weight = 0
for a, b in matching.items():
    weight += G[a][b]['weight']
-weight/2

In [ ]:
dataset_balanced = dataset.loc[matching.keys()]
len(dataset_balanced)

In [ ]:
dataset_balanced.head()

#### 5. Balancing the groups further

Based on your comparison of feature-value distributions from part 4, are you fully satisfied with your matching?
Would you say your dataset is sufficiently balanced?
If not, in what ways could the "balanced" dataset you have obtained still not allow you to draw valid conclusions?

Improve your matching by explicitly making sure that you match only subjects that have the same value for the problematic feature.
Argue with numbers and plots that the two groups (treated and control) are now better balanced than after part 4.

#### 6. A less naive analysis

Compare the outcomes (`re78`) between treated and control subjects, as you've done in part 1, but now only for the matched dataset you've obtained from part 5.
What do you conclude about the effectiveness of the job training program?

## Question 2: Applied ML

We are going to build a classifier of news to directly assign them to 20 news categories. Note that the pipeline that you will build in this exercise could be of great help during your project if you plan to work with text!

1. Load the 20newsgroup dataset. It is, again, a classic dataset that can directly be loaded using sklearn ([link](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html)).  
[TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf), short for term frequency–inverse document frequency, is of great help when if comes to compute textual features. Indeed, it gives more importance to terms that are more specific to the considered articles (TF) but reduces the importance of terms that are very frequent in the entire corpus (IDF). Compute TF-IDF features for every article using [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Then, split your dataset into a training, a testing and a validation set (10% for validation and 10% for testing). Each observation should be paired with its corresponding label (the article category).

2. Train a random forest on your training set. Try to fine-tune the parameters of your predictor on your validation set using a simple grid search on the number of estimator "n_estimators" and the max depth of the trees "max_depth". Then, display a confusion matrix of your classification pipeline. Lastly, once you assessed your model, inspect the `feature_importances_` attribute of your random forest and discuss the obtained results.